### 作業需求
請使用分類演算法將Google 商店評論文章做情緒分類：

1. 先將一Google 店家評論 (https://raw.githubusercontent.com/ywchiu/tibame_tm/master/data/google_comments.xlsx)  下載存入到自己電腦
2. 請使用Pandas 將該資料讀入名為 google 的 DataFrame 中?
3. 請利用Gensim 為該資料集建立一名為model的詞向量模型?
4. 請利用model求出該語料庫的句向量，並將句向量取名為 x, 預測目標取名為 y?
5. 程式碼能夠將資料(x,y)切分為訓練(train_x, train_y)與測試資料集(test_x, test_y)其中訓練資料集占80%, 測試資料集占20%(10%)
6. 請將訓練資料集 train_x 輸入至 SVM中，建立名為 clf 的分類模型(特徵為train_x，目標為 train_y)?
7. 請套用clf 到 測試資料集 test_x, 並產生預測結果 pred_y
7. 請根據 test_y, pred_y 計算出clf模型準確度(Accuracy)?


### 評分標準
1. 程式碼能順利運行 (10%)
2. 程式碼能順利讀取該檔案，並將資料讀成名為google 的DataFrame (5%)
3. 程式碼能將 content 欄位進行切詞，並將切詞後的資料放入語料庫 corpus 的 list 中 (5%)
4. 程式碼能使用 gensim 建立名為model 的詞向量模型 (20%)
5. 程式碼能使用模型model求出該語料庫的句向量，並將句向量取名為 x, 預測目標取名為 y? (20%)
6. 程式碼能夠將資料x,y 切分為訓練(train_x, train_y)與測試資料集(test_x, test_y)其中訓練資料集占80%, 測試資料集占20%(10%) 
7. 程式碼使用SVM建立名為clf的分類模型(10%)
8. 程式碼能套用clf 到 測試資料集 test_x, 並產生預測結果 pred_y (10%)
9. 程式碼能計算模型準確度(Accuracy) (10%)

In [1]:
!gdown https://raw.githubusercontent.com/ywchiu/tibame_tm/master/data/google_comments.xlsx

Downloading...
From: https://raw.githubusercontent.com/ywchiu/tibame_tm/master/data/google_comments.xlsx
To: /content/google_comments.xlsx
100% 65.1k/65.1k [00:00<00:00, 40.3MB/s]


In [2]:
import pandas as pd
import numpy as np
df = pd.read_excel('https://raw.githubusercontent.com/ywchiu/tibame_tm/master/data/google_comments.xlsx', index_col=0)

In [3]:
df.head()

,username,score,comment
0,倪依芙,5,起初找不到入口，還以為沒座位，但是原來入口在旁邊樓梯～座位區不大，但是料理很好吃～肉圓的醬微...
1,Mimi龍,4,聞名已久，今天特別來現場試試，目前改成點餐機點餐， ⋯⋯更多聞名已久，今天特別來現場試試，目...
2,林昌逸,1,非常生氣服務員的態度😠服務很糟糕，麵都還沒吃完，一起身就把麵收走，擺明趕客人😡😡😡
3,林幸蓁,5,內用與外帶有不同的點餐方式內用須至店內使用機台點菜、輸入桌號並付款外帶好像只需在入口處點餐即...
4,台灣TAXI市區叫車禮車包車約拍找小鄭,5,乾淨衛生，廁所很大在地下室，有免費飲料可以喝，拉麵愛好吃，麻辣豆腐不錯、肉粽不錯，目前吃過很...


In [4]:
def convert_score(score):
    if int(score) >= 4:
        status = 'good'
    elif int(score) <= 2:
        status = 'bad'
    else:
        status = 'norm'
    return status

In [6]:
df['status'] = df['score'].map(lambda e: convert_score(e))

In [7]:
df.head()

,username,score,comment,status
0,倪依芙,5,起初找不到入口，還以為沒座位，但是原來入口在旁邊樓梯～座位區不大，但是料理很好吃～肉圓的醬微...,good
1,Mimi龍,4,聞名已久，今天特別來現場試試，目前改成點餐機點餐， ⋯⋯更多聞名已久，今天特別來現場試試，目...,good
2,林昌逸,1,非常生氣服務員的態度😠服務很糟糕，麵都還沒吃完，一起身就把麵收走，擺明趕客人😡😡😡,bad
3,林幸蓁,5,內用與外帶有不同的點餐方式內用須至店內使用機台點菜、輸入桌號並付款外帶好像只需在入口處點餐即...,good
4,台灣TAXI市區叫車禮車包車約拍找小鄭,5,乾淨衛生，廁所很大在地下室，有免費飲料可以喝，拉麵愛好吃，麻辣豆腐不錯、肉粽不錯，目前吃過很...,good


In [10]:
corpus = []
tags = []

import jieba
for idx, item in df[df['status'].isin(['good', 'bad'])].iterrows():
    corpus.append(list(jieba.cut(item.get('comment'))))
    tags.append(item.get('status'))

Building prefix dict from the default dictionary ...
DEBUG:jieba:Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
DEBUG:jieba:Dumping model to file cache /tmp/jieba.cache
Loading model cost 1.229 seconds.
DEBUG:jieba:Loading model cost 1.229 seconds.
Prefix dict has been built successfully.
DEBUG:jieba:Prefix dict has been built successfully.


In [ ]:
import gensim
from gensim.models import Word2Vec
model = Word2Vec(corpus, size=100, min_count=1, window=5, workers=-1, iter=100)
print(model.vector_size)

In [26]:
vecs = []
y = []
for s, tag in zip(corpus, tags):
    vec = np.zeros(model.vector_size).reshape((model.vector_size,))
    count = 0
    for w in s:
        if w in model:
            vec += model.wv.get_vector(w)
            count += 1
    if count > 0:
        vecs.append(vec / count)
        y.append(tag)

# print(np.array(vecs).shape)
# print(vec.shape)
# print(model.wv.get_vector('讚').shape)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  import sys


In [27]:
import sklearn
from sklearn.model_selection import train_test_split
train_X, test_X, train_y, test_y = train_test_split(vecs, y, train_size=0.8, random_state=42)

In [30]:
from sklearn.svm import SVC
clf = SVC(kernel='rbf')
clf.fit(train_X, train_y)
pred_y = clf.predict(test_X)

In [32]:
from sklearn.metrics import accuracy_score, confusion_matrix
print(accuracy_score(test_y, pred_y))
print(clf.classes_)
print(confusion_matrix(test_y, pred_y))

0.7613636363636364
['bad' 'good']
[[29 11]
 [10 38]]
